# Моделирование и тестирование адаптивного планирования в реальном времени (Real-Time Scheduler)

**Цель:** Реагировать на события дня и моментально предлагать новый расклад расписания.


## 1. Импорт библиотек и подключение к базе данных PostgreSQL (medical_db)

In [ ]:
import os, json, logging, warnings
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from fastapi import FastAPI, WebSocket
import uvicorn, asyncio, aiokafka

# Настройки подключения к medical_db
DB_CONFIG = {
    'host': 'localhost',
    'database': 'medical_db',
    'user': 'postgres',
    'password': 'postgres',
    'port': '5432'
}
engine = create_engine(f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
print("Подключение к medical_db установлено")

## 2. Имитация событий (опоздание, неявка, экстренный случай) и загрузка расписания

In [ ]:
# Загрузка расписания на сегодня
query = """
SELECT 
    p.PLANNING_ID, p.DATE_CONS, p.HEURE, p.MEDECINS_CREATOR_ID, p.PATIENTS_ID,
    p.CANCELLED, p.CITO, p.STATUS, p.ARRIVE_DATE,
    m.SPECIALISATION_ID, m.FM_DEP_ID
FROM PLANNING p
LEFT JOIN MEDECINS m ON p.MEDECINS_CREATOR_ID = m.MEDECINS_ID
WHERE p.DATE_CONS = CURRENT_DATE
"""
df = pd.read_sql(query, engine)
print(f'Загружено {len(df)} записей на сегодня')
df.head()

## 3. Генерация событий (опоздание врача, неявка, экстренный случай)

In [ ]:
# Пример генерации событий
np.random.seed(42)
events = []
for idx, row in df.iterrows():
    if np.random.rand() < 0.05:
        events.append({'type': 'doctor_late', 'planning_id': row['PLANNING_ID']})
    if np.random.rand() < 0.03:
        events.append({'type': 'no_show', 'planning_id': row['PLANNING_ID']})
    if np.random.rand() < 0.01:
        events.append({'type': 'emergency', 'planning_id': row['PLANNING_ID']})
print(f'Сгенерировано {len(events)} событий')
events[:5]

## 4. Получение текущего состояния расписания (state-vector) и анализ очереди ожидания

In [ ]:
# Пример анализа очереди ожидания
waiting_queue = df[df['STATUS'] == 'waiting']
print(f'В очереди {len(waiting_queue)} пациентов')
waiting_queue.head()

## 5. Алгоритм пере-расписания: минимизация задержек, приоритеты (CITO, VIP)

In [ ]:
# Эвристика: сортировка очереди по приоритету и времени
waiting_queue = waiting_queue.copy()
waiting_queue['priority'] = waiting_queue['CITO'].map({'Y': 2, 'N': 1}).fillna(1)
waiting_queue = waiting_queue.sort_values(['priority', 'HEURE'], ascending=[False, True])
print('Топ-5 пациентов по приоритету:')
waiting_queue.head()

## 6. Корректировка расписания и сохранение снэпшотов

In [ ]:
# Пример корректировки: перенос времени приема
for event in events:
    if event['type'] == 'doctor_late':
        # Переносим все приемы врача на +15 минут
        doctor_id = df.loc[df['PLANNING_ID'] == event['planning_id'], 'MEDECINS_CREATOR_ID'].values[0]
        mask = df['MEDECINS_CREATOR_ID'] == doctor_id
        df.loc[mask, 'HEURE'] = pd.to_datetime(df.loc[mask, 'HEURE']) + timedelta(minutes=15)
    if event['type'] == 'no_show':
        # Освобождаем слот, добавляем пациента из очереди
        pass  # Здесь логика добавления резервного пациента
    if event['type'] == 'emergency':
        # Вставка экстренного пациента
        pass
print('Корректировка расписания завершена')

## 7. Сохранение снэпшотов и анализ отклонений

In [ ]:
# Сохраняем снэпшот изменений
snapshot = df[['PLANNING_ID', 'HEURE']].copy()
snapshot['SNAP_ID'] = datetime.now().strftime('%Y%m%d%H%M%S')
snapshot['DATE_CHANGE'] = datetime.now()
snapshot.to_csv('pl_snapshot.csv', index=False)
print('Снэпшот расписания сохранён в pl_snapshot.csv')